# PyJQuants Quickstart

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/obichan117/pyjquants/blob/main/docs/examples/quickstart.ipynb)

This notebook demonstrates the basic usage of PyJQuants - a yfinance-style Python library for [J-Quants API](https://jpx.gitbook.io/j-quants-en).

## Contents

1. [Setup](#1-setup)
2. [Single Ticker](#2-single-ticker)
3. [Price History](#3-price-history)
4. [Multi-Ticker Download](#4-multi-ticker-download)
5. [Market Indices](#5-market-indices)
6. [Next Steps](#6-next-steps)

## 1. Setup

### Install PyJQuants

In [ ]:
# Install PyJQuants (auto-detects Colab)
try:
    import pyjquants
    print(f'PyJQuants already installed (v{pyjquants.__version__})')
except ImportError:
    print('Installing PyJQuants...')
    !pip install -q pyjquants
    print('✓ Installed!')

### Configure API Key

You need a J-Quants account. [Sign up here](https://application.jpx-jquants.com/) (free tier available).

Get your API key from the [J-Quants dashboard](https://application.jpx-jquants.com/), then run the cell below.

**Option 1 (Recommended for Colab):** Use Colab's Secrets manager:
1. Click the 🔑 key icon in the left sidebar
2. Add a secret named `JQUANTS_API_KEY` with your API key
3. Toggle "Notebook access" ON

**Option 2:** Paste your API key directly when prompted below.

In [ ]:
import os

# Try multiple sources for API key
api_key = None

# 1. Check environment variable (already set)
if os.environ.get('JQUANTS_API_KEY'):
    api_key = os.environ['JQUANTS_API_KEY']
    print('✓ API key found in environment')

# 2. Try Colab secrets (if running in Colab)
if not api_key:
    try:
        from google.colab import userdata
        api_key = userdata.get('JQUANTS_API_KEY')
        os.environ['JQUANTS_API_KEY'] = api_key
        print('✓ API key loaded from Colab Secrets')
    except:
        pass

# 3. Prompt user to paste API key
if not api_key:
    print('=' * 50)
    print('Paste your J-Quants API key below and press Enter:')
    print('(Get it from: https://application.jpx-jquants.com/)')
    print('=' * 50)
    api_key = input('API Key: ').strip()
    
    if api_key:
        os.environ['JQUANTS_API_KEY'] = api_key
        print('✓ API key set!')
    else:
        print('✗ No API key provided. Please set JQUANTS_API_KEY.')

# Verify
if os.environ.get('JQUANTS_API_KEY'):
    print(f"\nAPI key configured (ends with ...{os.environ['JQUANTS_API_KEY'][-4:]})")

### Import the Library

In [ ]:
import pyjquants as pjq

print(f'PyJQuants version: {pjq.__version__}')

## 2. Single Ticker

Create a `Ticker` object using the stock code. This follows the yfinance API style.

In [ ]:
# Create a ticker object (Toyota)
ticker = pjq.Ticker('7203')

# Access stock information (lazy-loaded from API)
print(f'Code: {ticker.info.code}')
print(f'Name: {ticker.info.name}')
print(f'English Name: {ticker.info.name_english}')
print(f'Sector: {ticker.info.sector}')
print(f'Market: {ticker.info.market}')

## 3. Price History

Use the `.history()` method to get price data, just like yfinance.

### Recent Prices

In [ ]:
# Get recent 30 days of price data (default)
df = ticker.history('30d')

print(f'Data points: {len(df)}')
print(f'Columns: {list(df.columns)}')
print()
df[['date', 'open', 'high', 'low', 'close', 'volume']].tail()

### Different Time Periods

In [ ]:
# Various period formats
df_1y = ticker.history('1y')     # 1 year
df_6mo = ticker.history('6mo')   # 6 months
df_1w = ticker.history('1w')     # 1 week

print(f'1 year: {len(df_1y)} days')
print(f'6 months: {len(df_6mo)} days')
print(f'1 week: {len(df_1w)} days')

### Custom Date Range

In [ ]:
from datetime import date

# Get prices for a specific date range
df_custom = ticker.history(start='2024-01-01', end='2024-06-30')

# Or using date objects
df_custom = ticker.history(start=date(2024, 1, 1), end=date(2024, 6, 30))

print(f'Custom range: {len(df_custom)} trading days')
df_custom[['date', 'close']].head(10)

### Visualize Prices

In [ ]:
import matplotlib.pyplot as plt

# Get 1 year of data for visualization
df = ticker.history('1y')

fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(df['date'], df['close'], 'b-', linewidth=2)
ax.set_title(f'{ticker.info.name} ({ticker.code}) - Stock Price', fontsize=14)
ax.set_xlabel('Date')
ax.set_ylabel('Close Price (JPY)')
ax.grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 4. Multi-Ticker Download

Download price data for multiple tickers at once using `pjq.download()`.

In [ ]:
# Download close prices for multiple tickers
codes = ['7203', '6758', '7974', '9984']  # Toyota, Sony, Nintendo, SoftBank

df_multi = pjq.download(codes, period='30d')

print(f'Shape: {df_multi.shape}')
df_multi.tail()

In [ ]:
# Plot multiple stocks
fig, ax = plt.subplots(figsize=(12, 6))

for code in codes:
    ax.plot(df_multi['date'], df_multi[code], label=code, linewidth=2)

ax.set_title('Stock Price Comparison', fontsize=14)
ax.set_xlabel('Date')
ax.set_ylabel('Close Price (JPY)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 5. Market Indices

Access market indices like TOPIX and Nikkei 225.

In [ ]:
# Get TOPIX index
topix = pjq.Index.topix()
print(f'Index: {topix.name}')

# Get TOPIX history
df_topix = topix.history('30d')
df_topix[['date', 'close']].tail()

In [ ]:
# Get Nikkei 225 index
nikkei = pjq.Index.nikkei225()
df_nikkei = nikkei.history('30d')

print(f'Nikkei 225 latest close: {df_nikkei["close"].iloc[-1]:,.0f}')

## 6. Next Steps

You've learned the basics of PyJQuants! Here are some next steps:

### Search for Tickers

```python
# Search by name
tickers = pjq.search('toyota')
for t in tickers:
    print(f'{t.code}: {t.info.name}')
```

### Financial Data

```python
# Get financial statements
financials = ticker.financials

# Get dividend history  
dividends = ticker.dividends
```

### Market Utilities

```python
from datetime import date

market = pjq.Market()

# Check trading days
market.is_trading_day(date(2024, 12, 25))  # False (Christmas)

# Get sector list
sectors = market.sectors_33
```

### Resources

- [PyJQuants Documentation](https://obichan117.github.io/pyjquants)
- [GitHub Repository](https://github.com/obichan117/pyjquants)
- [J-Quants API Documentation](https://jpx.gitbook.io/j-quants-en/)